In [29]:
'''The idea here is to build a set of functions that I can use to flatten several columns of the kenya dataset. 
My plan is to build a function for each column that will take each row value in succession (by using .apply()) and  
produce one or more lists, which can then be added as columns in the kenya dataframe.  In what I have below, I wanted
to simply start to extract values from a list of dicts that make up the row values in the column u'borrowers'. The
list in each row of this colunm can be longer than one (1) when there is more than one borrower.  So, I wanted 
to use a recursive function that could process until the list had been completed.  Perhaps this was a mistake.
I got something to work, but the code doesn't work the way I wanted. 
First, what I want is for each row a list like ['M','M','F','M'] that shows the gender of the borrowers.
Then I want to take the list and generate: total number borrowers
                                           proportion that are female
With those variables I can generate several others.

There are several issues:
1. the passing to the recursive function of a global list variable is a problem
2. The way I've set things up, even the original DataFrame gets altered in the process. I know that this is because
I don't really understand when something like v1=df.iloc[:,3] ends up acting like a pointer to the values in df,
instead of being a series.  So I end up screwing up the original datasets through treating = as asignment, instead
of as a pointer.  If you could give me some guidance here, it would help alot.

I have made a bunch of notes in the notebook, below

--Peter '''


SyntaxError: invalid syntax (<ipython-input-29-f55e428969b0>, line 2)

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import os
import json
from pandas.io.json import json_normalize
import cPickle as pk
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [4]:
handle=open('../data/Kenya.pickle','r')
df2=pk.load(handle)
handle.close()
# comment line 2

In [25]:
df2.shape

(98099, 47)

In [26]:
df2.columns

Index([                     u'activity',                 u'basket_amount',
            u'bonus_credit_eligibility',                     u'borrowers',
                             u'country',                  u'country_code',
       u'currency_exchange_loss_amount',                    u'delinquent',
                         u'description',                 u'funded_amount',
                         u'funded_date',                     u'geo.level',
                           u'geo.pairs',                      u'geo.type',
                                  u'id',                         u'image',
                      u'journal_totals',                     u'languages',
                        u'lender_count',                   u'loan_amount',
                            u'location',                          u'name',
                                 u'obs',                   u'paid_amount',
                           u'paid_date',                    u'partner_id',
                         

In [8]:
df3=df2

In [27]:
df3.iloc[0,3]

[{u'first_name': u'Victor',
  u'gender': u'M',
  u'last_name': u'',
  u'pictured': True}]

In [10]:
recipients=df3.iloc[:,3]
#recipients
def length(x):
    return len(x)
num_recip=recipients.apply(length)#.value_counts().head(10)
test=recipients[(num_recip==2)]#.iloc[0]


In [11]:
len(test)        # after doing some tests below, and then running this line again, I find the value decreases
                 # I am not clear on why this happens, except that I know that '=' doesn't really mean assignment
                 # in python.  I'm simply not writing the my code in a way to use this behavior, or avoid its 
                 # consequences... What ever I am doing below is altering df2 and df3

619

In [12]:
test.iloc[0:10] # I started taking a single observation of this, something like test1=test.iloc[1],
                #to use for development, but I observed that all of 'test' was becoming modified.
                # In fact, it seemed as though a manipulation of test1 in this case changed test, which
                # changed 'recipients', which changed 'df3, etc, etc.
                # How could I have taken a single row of 'test' to use in function development but not
                # alter the original Series??

344    [{u'gender': u'F', u'first_name': u'Sidi', u'l...
272    [{u'gender': u'F', u'first_name': u'Sidi', u'l...
338    [{u'gender': u'M', u'first_name': u'Bendera', ...
416    [{u'gender': u'F', u'first_name': u'Sidi', u'l...
236    [{u'gender': u'M', u'first_name': u'Nyevu', u'...
237    [{u'gender': u'F', u'first_name': u'Thabu', u'...
499    [{u'gender': u'M', u'first_name': u'Jacob', u'...
0      [{u'gender': u'M', u'first_name': u'Jacob', u'...
109    [{u'gender': u'M', u'first_name': u'Mdigo ', u...
437    [{u'gender': u'M', u'first_name': u'Jumwa', u'...
Name: borrowers, dtype: object

In [13]:
test1=test

In [14]:
recip=test.iloc[0]

In [15]:
recip

[{u'first_name': u'Sidi',
  u'gender': u'F',
  u'last_name': u'',
  u'pictured': True},
 {u'first_name': u'Jumwa',
  u'gender': u'F',
  u'last_name': u'',
  u'pictured': True}]

In [17]:
                               # here is a test list of dicts to simulate an entry in df2.borrowers
                            # I started using this because of the unwanted alterations I was causing to df2 and df3
dat1=[{u'first_name': u'Sidi',
  u'gender': u'F',
  u'last_name': u'',
  u'pictured': True},
 {u'first_name': u'Jumwa',
  u'gender': u'F',
  u'last_name': u'',
  u'pictured': True}]

In [18]:
lst=[]                        #a globally defined list to fill with the values of genders as given for each
                              # entry in df2.borrowers

In [19]:
                              # a recursive function to extract the gender values from each entry in df2.borrowers 
                              # and fill the list 'lst'. This function would be used in a wrapper to grow the lists
                              # mentioned above, which would be incorporated into the dataset as new columns.
            
                              # I have been trying to write this without the globally-defined variable 'lst', and have
                              # the function return a list, but I haven't been able to get that to work.  Until now
                              # I have had to use the globally-defined 'lst'.
def gender_extract(recip,lst):
    if len(recip)==1:
        lst.append(recip.pop()['gender'])
        print 'now on finish'
        at_last=lst
        return at_last
    else:
        cat=recip.pop()
        lst.append(cat['gender'])
        print 'cat = ',cat,'and lst = ',lst
    gender_extract(recip,lst)


In [20]:
                               #so here I feed dat1 to the function.  But what I really wanted to do was to be able to 
                               #give it a subset of df3, which I wanted to function as a copy of df2.  However, when
                               #I did that, both df2 and df3 were altered.  If df3 got chewed up, that would be ok, if
                               # it the copy of df2 remained unaltered.  
                               #The idea would be to use the function gender_extract inside
                               # another function that would put together new lists, which I would then integrate as
                               # new columns in df2.
                               # Then I would go and write similar functions for the other columns that need flattening
                               #eventually have a flattened df2
                             


result=gender_extract(dat1,lst)

cat =  {u'gender': u'F', u'first_name': u'Jumwa', u'last_name': u'', u'pictured': True} and lst =  [u'F']
now on finish


In [21]:
print result   # I tried to make the function return something, but I only get a value of None

None


In [23]:
print lst         #so the variable 'lst' has the correct values, but I am stuck with a globally-defined variable

 [u'F', u'F']


In [24]:
dat1              # dat1 is empty, which is expected because I did .pop in the recursion.  However, when I defined
                  # slices of df3 and used them, df3 and df2 also became altered!!! I am clearly not understanding something
                  # very important

[]

In [ ]:
# things below here were just used to get parts of gender_extract() to work.

In [431]:
lst=[]
if len(recip)==1:
        lst.append(recip.pop()['gender'])
        print lst
else:
    print lst,recip

[] [{u'gender': u'F', u'first_name': u'Sidi', u'last_name': u'', u'pictured': True}, {u'gender': u'F', u'first_name': u'Sidi', u'last_name': u'', u'pictured': True}]


In [432]:
recip

[{u'first_name': u'Sidi',
  u'gender': u'F',
  u'last_name': u'',
  u'pictured': True},
 {u'first_name': u'Sidi',
  u'gender': u'F',
  u'last_name': u'',
  u'pictured': True}]